<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.0.3"><span class="toc-item-num">2.0.3&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.0.4"><span class="toc-item-num">2.0.4&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#XGBClassifier" data-toc-modified-id="XGBClassifier-2.0.5"><span class="toc-item-num">2.0.5&nbsp;&nbsp;</span>XGBClassifier</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install -q lightgbm

Unable to create process using 'C:\ProgramData\Anaconda3\envs\ds_practicum_env\python.exe C:\ProgramData\Anaconda3\envs\ds_practicum_env\Scripts\pip-script.py install -q lightgbm'


In [2]:
!pip install -q xgboost

Unable to create process using 'C:\ProgramData\Anaconda3\envs\ds_practicum_env\python.exe C:\ProgramData\Anaconda3\envs\ds_practicum_env\Scripts\pip-script.py install -q xgboost'


In [3]:
!pip install -q Pipeline 

Unable to create process using 'C:\ProgramData\Anaconda3\envs\ds_practicum_env\python.exe C:\ProgramData\Anaconda3\envs\ds_practicum_env\Scripts\pip-script.py install -q Pipeline'


In [4]:
!pip install -q nltk

Unable to create process using 'C:\ProgramData\Anaconda3\envs\ds_practicum_env\python.exe C:\ProgramData\Anaconda3\envs\ds_practicum_env\Scripts\pip-script.py install -q nltk'


In [1]:
# импортируем все необходимое
import pandas as pd
import numpy as np 
import re

#from pymystem3 import Mystem

import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import stopwords 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...


In [2]:
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
stop_words = set(stopwords.words('english'))

In [4]:
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('toxic_comments.csv')
    
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
display(df['toxic'].value_counts())

0    143106
1     16186
Name: toxic, dtype: int64

In [7]:
class_ratio = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]
class_ratio

8.841344371679229

Классы не сбалансированны. Учтем это при обучении моделей.

Упростим текст для извлечения признаков.

Этапы предобработки текста:

Токенизация (англ. tokenization) — разбиение текста на токены: отдельные фразы, слова, символы.
Лемматизация (англ. lemmatization) — приведение слова к начальной форме (лемме).

Напишем функцию для токенизации и лемматизации текста:

In [26]:
lemmatizer = WordNetLemmatizer()

def lemm_text(text):
    text = re.sub(r'[^a-zA-z ]', ' ', text)
    text = text.lower()
    token = nltk.word_tokenize(text)
    text = [word for word in token if word not in stop_words]
    text = [lemmatizer.lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

df['text'] = df['text'].apply(lemm_text)

In [27]:
df.head()

,Unnamed: 0,text,toxic
0,0,explanation edits made username hardcore metal...,0
1,1,aww match background colour seemingly stuck th...,0
2,2,hey man really trying edit war guy constantly ...,0
3,3,make real suggestion improvement wondered sect...,0
4,4,sir hero chance remember page,0


Вывод: данные обработаны, очищены от лишних символов, лемматизированы.

## Обучение

In [28]:
RANDOM_STATE = np.random.RandomState(12345)

Разделим данные на обучающую и тестовую выборки

In [29]:
X = df.drop('toxic', axis=1)
y = df['toxic']

In [61]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.1, random_state = RANDOM_STATE, stratify = y)


In [62]:
print('Train data shape:', X_train.shape, y_train.shape)

Train data shape: (143362, 2) (143362,)


In [63]:
print('Test data shape:', X_test.shape, y_test.shape)

Test data shape: (15930, 2) (15930,)


Напишем функцию для поиска лучших гиперпараметров используя RandomizedSearchCV с кросс-валидацией на 5-ти подвыборках и расчетом TF-IDF для каждой выборки.

In [64]:
def train_model(model, params):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(min_df = 1)),
        ('model', model)])
    grid = RandomizedSearchCV(estimator = pipeline, param_distributions = params,
                              cv = 5, scoring = 'f1',
                              n_jobs = -1, refit = True,
                             random_state = RANDOM_STATE)
    grid.fit(X_train['text'], y_train)
    print('Лучший результат F1:', grid.best_score_)
    print('Лучшие параметры модели:', grid.best_params_)
    return grid

#### LogisticRegression

In [65]:
lr = train_model(LogisticRegression(random_state = RANDOM_STATE, class_weight = 'balanced'), {
    'model__C': [0.1, 1.0, 10.0],
    'model__penalty': ['l2']})

Лучший результат F1: 0.7675270409603879
Лучшие параметры модели: {'model__penalty': 'l2', 'model__C': 10.0}


#### DecisionTreeClassifier

In [66]:
dt = train_model(DecisionTreeClassifier(random_state = RANDOM_STATE, class_weight = 'balanced'), {
    'model__criterion':['gini','entropy'],
    'model__max_depth':[2,4,6]})

Лучший результат F1: 0.5028559437500981
Лучшие параметры модели: {'model__max_depth': 6, 'model__criterion': 'entropy'}


#### RandomForestClassifier

In [67]:
rf = train_model(RandomForestClassifier(random_state = RANDOM_STATE, class_weight = 'balanced'), {
    'model__max_depth':[5,15],
    'model__max_features' : list(range(1,20, 2))})

Лучший результат F1: 0.2875242298979237
Лучшие параметры модели: {'model__max_features': 19, 'model__max_depth': 15}


#### LGBMClassifier

In [68]:
lgbm = train_model(
    lgb.LGBMClassifier(random_state = RANDOM_STATE, class_weight = 'balanced'), {
    'model__max_depth':[2,10,15],
    'model__learning_rate': [0.1, 0.3]})

Лучший результат F1: 0.7576468883353134
Лучшие параметры модели: {'model__max_depth': 15, 'model__learning_rate': 0.3}


#### XGBClassifier

In [69]:
import xgboost as xgb

In [70]:
xgb = train_model(
    xgb.XGBClassifier(random_state = RANDOM_STATE, booster = 'gbtree', class_weight = 'balanced'), {
        'model__learning_rate': [0.5, 1.0]})

[14:04:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "class_weight" } are not used.

Лучший результат F1: 0.7550916350588797
Лучшие параметры модели: {'model__learning_rate': 0.5}


Вывод:
    
    Лучший результат на тренировочной выборке показала модель LogisticRegression:
        
        F1: 0.7675270409603879
        
        Лучшие параметры модели: {'model__penalty': 'l2', 'model__C': 10.0}
        

### Тестирование модели

Напишем такую же функцию для обучения и расчета F1 на тестовой выборке

In [72]:
from sklearn.feature_extraction.text import CountVectorizer

In [73]:
# Создаем объект CountVectorizer
vectorizer = CountVectorizer()

# Преобразуем текстовые данные в числовые
X_train_vectorized = vectorizer.fit_transform(X_train['text'])
X_test_vectorized = vectorizer.transform(X_test['text'])

# Обучаем модель
lr_test = LogisticRegression(penalty='l2', C=10.0, random_state=RANDOM_STATE, class_weight='balanced')
lr_test.fit(X_train_vectorized, y_train)

# Делаем предсказания на тестовом наборе данных
y_pred = lr_test.predict(X_test_vectorized)
f1_test = f1_score(y_test, y_pred)

print('Лучший результат F1:', f1_test)

Лучший результат F1: 0.758004038073262


F1 лучшей модели LogisticRegression на тестовой выборке показал резульат 0.758. Уровень метрики соответствует требованиям, значит цель достугнута.

## Выводы

1. Мы получили набор данных с разметкой о токсичности правок.

Подготовили данные. Выполнили:

Токенизацию (англ. tokenization) — разбиение текста на токены: отдельные фразы, слова, символы.

Лемматизацию (англ. lemmatization) — приведение слова к начальной форме (лемме).

2. Обучили 5 моделей классификации.

Лучший результат на тренировочной выборке показала модель LogisticRegression:

    F1: 0.7675270409603879

    Лучшие параметры модели: {'model__penalty': 'l2', 'model__C': 10.0}
    
3. Уровень метрики F1 для лучшей модели LogisticRegression на тестовой выборке показал резульат 0.758.